In [1]:
import glob
import zipfile
import json
import pandas as pd
import pickle
from pprint import pprint

In [2]:
data_path = "/home/singh_shruti/data/mag_aminer/"

## Count references in the whole dataset

In [13]:
with open('mag_papers.pickle', 'rb') as handle:
    all_mag_papers = pickle.load(handle)

In [15]:
total_papers = 0
total_refs = 0
ref_counts = 0

for v in all_mag_papers.values():
#     print(v)
#     break
    total_papers += 1
    if "references" in v and v["references"]:
        total_refs += len(v["references"])
        ref_counts += 1

In [16]:
total_papers, total_refs, ref_counts

(23873092, 0, 0)

## Filtering subsests

In [3]:
paper_files = glob.glob(data_path + "mag_papers_[0-9].zip")
author_files = glob.glob(data_path + "mag_authors_[0-9].zip")

In [4]:
paper_files, author_files

(['/home/singh_shruti/data/mag_aminer/mag_papers_2.zip',
  '/home/singh_shruti/data/mag_aminer/mag_papers_1.zip',
  '/home/singh_shruti/data/mag_aminer/mag_papers_0.zip'],
 ['/home/singh_shruti/data/mag_aminer/mag_authors_2.zip',
  '/home/singh_shruti/data/mag_aminer/mag_authors_0.zip',
  '/home/singh_shruti/data/mag_aminer/mag_authors_1.zip'])

In [7]:
mag_papers = {}

later_2016 = 0
venue_missing = 0
till_2016 = 0
references_present = 0

for f in paper_files[1:2]:
    archive = zipfile.ZipFile(f, 'r')
    zip_files = archive.namelist()
    print(zip_files)
    for fi in zip_files:
        bytes_text = archive.read(fi)
        print("Read file {} in archive {}".format(fi, zip_files))
        try:
            str_list = bytes_text.decode('utf-8').split("\n")
            print("Length of file: ", len(str_list))
            idx = 0
            
            for l in str_list:
                if idx % 1000000 == 0:
                    print(idx)
                idx += 1
                try:
                    paper_dict = json.loads(l)
                    if "year" in paper_dict and paper_dict["year"]:
                        if paper_dict["year"] < 2017:
                            till_2016 = till_2016 + 1
                            mag_papers[paper_dict["id"]] = paper_dict
                            if "references" in paper_dict and paper_dict["references"]:
                                references_present += 1
                        else:
                            later_2016 = later_2016 + 1
                    else:
                        venue_missing = venue_missing + 1
                except Exception as ex:
                    print("err for json loading")
                    print(ex)
        except Exception as ex:
            print("err for bytes decoding")
            print(ex)
        break

['mag_papers_4.txt', 'mag_papers_5.txt', 'mag_papers_6.txt', 'mag_papers_7.txt']
Read file mag_papers_4.txt in archive ['mag_papers_4.txt', 'mag_papers_5.txt', 'mag_papers_6.txt', 'mag_papers_7.txt']
Length of file:  21477822
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
err for json loading
Expecting value: line 1 column 1 (char 0)


In [12]:
total_papers = 0
total_refs = 0
ref_counts = 0

uniq_keys = []

for v in mag_papers.values():
    uniq_keys = uniq_keys + list(v.keys())
    uniq_keys = list(set(uniq_keys))
#     total_papers += 1
#     if "references" in v and v["references"]:
#         total_refs += len(v["references"])
#         ref_counts += 1

In [13]:
uniq_keys

['authors',
 'venue',
 'year',
 'doi',
 'n_citation',
 'volume',
 'title',
 'publisher',
 'id',
 'issue',
 'page_start',
 'doc_type',
 'page_end']

In [15]:
ls /home/singh_shruti/data/mag_aminer/

mag_authors_0.zip  mag_papers_0.zip  mag_venues.zip
mag_authors_1.zip  mag_papers_1.zip  paper_linking_pairs.zip
mag_authors_2.zip  mag_papers_2.zip  venue_linking_pairs.zip


## Read linking file

In [22]:
archive = zipfile.ZipFile(data_path+'venue_linking_pairs.zip', 'r')
zip_files = archive.namelist()
print(zip_files)
for fi in zip_files:
    bytes_text = archive.read(fi)
#     try:
#         str_list = bytes_text.decode('utf-8').split("\n")
#         for l in str_list:
#                 if idx % 1000000 == 0:
#                     print(idx)
#                 idx += 1
#                 try:
#                     paper_dict = json.loads(l)
#     except Exception as ex:
#         print(ex)

['venue_linking_pairs.txt']


In [23]:
bytes_text[0:100]

b'{"mid": "5bf573b81c5a1dcdd96ec669", "aid": "5451a5c9e0cf0b02b5f3bd3c"}\n{"mid": "5bf573b81c5a1dcdd96e'

#### Not useful

## Read SS data

In [25]:
ls /opt/datasets/semanticscholar/

license.txt       s2-corpus-044.gz  s2-corpus-090.gz  s2-corpus-141.gz
manifest.txt      s2-corpus-045.gz  s2-corpus-091.gz  s2-corpus-142.gz
s2-corpus-000.gz  s2-corpus-046.gz  s2-corpus-092.gz  s2-corpus-143.gz
s2-corpus-001.gz  s2-corpus-047.gz  s2-corpus-093.gz  s2-corpus-144.gz
s2-corpus-002.gz  s2-corpus-048.gz  s2-corpus-094.gz  s2-corpus-145.gz
s2-corpus-003.gz  s2-corpus-049.gz  s2-corpus-095.gz  s2-corpus-146.gz
s2-corpus-004.gz  s2-corpus-050.gz  s2-corpus-096.gz  s2-corpus-147.gz
s2-corpus-005.gz  s2-corpus-051.gz  s2-corpus-097.gz  s2-corpus-148.gz
s2-corpus-006.gz  s2-corpus-052.gz  s2-corpus-098.gz  s2-corpus-149.gz
s2-corpus-007.gz  s2-corpus-053.gz  s2-corpus-099.gz  s2-corpus-150.gz
s2-corpus-008.gz  s2-corpus-054.gz  s2-corpus-100.gz  s2-corpus-151.gz
s2-corpus-009.gz  s2-corpus-055.gz  s2-corpus-101.gz  s2-corpus-152.gz
s2-corpus-010.gz  s2-corpus-056.gz  s2-corpus-102.gz  s2-corpus-153.gz
s2-corpus-011.gz  s2-corpus-057.gz  s2-corpus-103.gz  s2-corpus-154.gz
s2-cor

In [27]:
import gzip

with gzip.open('/opt/datasets/semanticscholar/s2-corpus-183.gz', 'rb') as f:
     file_content = f.read()

In [28]:
type(file_contentnt)

bytes

In [30]:
file_content[0:1000]

b'{"entities":[],"journalVolume":"","journalPages":"","pmid":"","fieldsOfStudy":["Geography"],"year":2008,"outCitations":[],"s2Url":"https://semanticscholar.org/paper/75eaab074bb97099f8d58da6a551eb4338f9781f","s2PdfUrl":"","id":"75eaab074bb97099f8d58da6a551eb4338f9781f","authors":[{"name":"Thomas Chevau","ids":["104559805"]},{"name":"Alfred Bastin","ids":["104586948"]},{"name":"Jean-Marie Halleux","ids":["89612773"]},{"name":"J. L. Lambotte","ids":["116428825"]}],"journalName":"","paperAbstract":"","inCitations":[],"pdfUrls":[],"title":"Analyse statistique des entreprises pr\xc3\xa9sentes dans les parcs d\xe2\x80\x99activit\xc3\xa9s \xc3\xa9conomiques sous l\xe2\x80\x99angle de leur compatibilit\xc3\xa9 avec l\xe2\x80\x99habitat. In: Identification des localisations optimales des activit\xc3\xa9s \xc3\xa9conomiques","doi":"","sources":[],"doiUrl":"","venue":""}\n{"entities":[],"journalVolume":"37 2","journalPages":"\\n          257-272\\n        ","pmid":"31551647","fieldsOfStudy":["Ps